In [14]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import numpy as py
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random


In [16]:
df= pd.read_csv('/content/drive/MyDrive/clickbait dataset/clickbait_data.csv')

In [17]:
print(df.head())

                                            headline  clickbait
0                                 Should I Get Bings          1
1      Which TV Female Friend Group Do You Belong In          1
2  The New "Star Wars: The Force Awakens" Trailer...          1
3  This Vine Of New York On "Celebrity Big Brothe...          1
4  A Couple Did A Stunning Photo Shoot With Their...          1


In [18]:
df.sample(5)

,headline,clickbait
18356,Immigration Accord by Labor Boosts Obama Effort,0
4302,We Know Who You Plan On Spending Valentine's D...,1
29320,Underwater volcano erupts in Pacific near Japan,0
26328,"How Safe Is That Mutual Fund Next Egg, Anyhow?",0
21061,Canadian politician calls for terrorism inquir...,0


In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

In [20]:
Xtr_str, Xts_str, ytr, yts = train_test_split(
    df['headline'].values, df['clickbait'].values, shuffle=True, random_state=0, test_size=0.25
)

In [21]:
#  text to numerical using CountVectorizer
vec = CountVectorizer(stop_words='english')
Xtr_vec = vec.fit_transform(Xtr_str)
Xts_vec = vec.transform(Xts_str)
print(Xtr_vec.shape)


(24000, 19769)


In [22]:
# RandomForestClassifier
rf = RandomForestClassifier(n_estimators=5, random_state=0)
rf.fit(Xtr_vec, ytr)  # Train on training data


yts_rf = rf.predict(Xts_vec)

# accuracy
acc_rf = accuracy_score(yts, yts_rf)
print("Random Forest Accuracy:", acc_rf)


Random Forest Accuracy: 0.90475


In [23]:
# PCA
n_comp_list = py.arange(1, 20, 2)
n_fold = 3
score_val = py.zeros((len(n_comp_list), n_fold))

kf = KFold(n_splits=n_fold, shuffle=False)

for i, n_comp in enumerate(n_comp_list):
    for j, (train_idx, val_idx) in enumerate(kf.split(Xtr_vec)):
        # Apply PCA
        pca = PCA(n_components=n_comp, random_state=0)
        Xtr_pca = pca.fit_transform(Xtr_vec[train_idx].toarray())
        Xval_pca = pca.transform(Xtr_vec[val_idx].toarray())

        # Train and evaluate Random Forest
        rf = RandomForestClassifier(n_estimators=5, random_state=0)
        rf.fit(Xtr_pca, ytr[train_idx])
        yval_pred = rf.predict(Xval_pca)
        score_val[i, j] = accuracy_score(ytr[val_idx], yval_pred)

# error
acc_mean = score_val.mean(axis=1)
acc_se = score_val.std(axis=1) / py.sqrt(n_fold)
print("Mean accuracy:", acc_mean)
print("Standard error:", acc_se)


Mean accuracy: [0.79116667 0.8635     0.87666667 0.88904167 0.89041667 0.89704167
 0.8965     0.89845833 0.896      0.899375  ]
Standard error: [0.00493863 0.00214411 0.00156162 0.00208028 0.00267274 0.00290673
 0.00182954 0.00301462 0.00167809 0.00214168]
